Title:

Classification of Red Wine Quality 

Introduction:

In our classification, we will be using the following variables to create our classification model: [ITEMS]

Firstly, alcohol will be used, as ethanol as a compound is known to impact individuals perception of wine flavor, thus influencing percieved quality (Caballero & Segura, 2017). 


Sources:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5328826/#:~:text=First%2C%20for%20the%20wine%20industry,sluggish%20or%20even%20stuck%20fermentations.


Preliminary Exploratory Data Analysis:



Methods:



Expected Outcomes and Significance:



Data Source:
https://archive.ics.uci.edu/ml/datasets/wine+quality